In [1]:
# Import libraries 
import os
#os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

#os.environ['KMP_DUPLICATE_LIB_OK']='True'

import torch 

import pickle 
import matplotlib.pyplot as plt

import numpy as np

import cv2

import  time
import glob

from scipy.io import loadmat

import random
from random import seed, randint

from sklearn.model_selection import train_test_split

import pandas as pd

from imutils.video import FPS

import imutils

from torch.utils.data import DataLoader, Dataset

from threading import Thread
import sys

from queue import Queue

from imutils.video import FileVideoStream

import numpy as np
import argparse
import imutils
import time
import pytorchvideo 

In [ ]:
#%% if necessary
!pip install torchsummary

In [2]:
#%% Prepare Data Loader ???

import importlib
import data_set_loader
importlib.reload(data_set_loader)
from data_set_loader import data_set_Prep
from data_set_loader import brightness_augment, snp_RGB, test_data

#%% 
import losses_PT
importlib.reload(losses_PT)

optimizer = torch.optim.Adam(my_net.parameters(), lr=1e-4)

from losses_PT import TripletLoss1, InfoNceLoss, NT_Xent, InfoNCE, align_uniform
L= InfoNceLoss(0.5)

tan_m =  nn.Tanh()


# model_name = "slowfast_r50"
# model = torch.hub.load("facebookresearch/pytorchvideo", model=model_name, pretrained=False)

In [3]:
def set_deterministic(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed) 
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.enabled = False

set_deterministic(seed=42)

In [4]:
#%% Data Directory

def get_files(path_dir = '../Data/Pickle_files'):
    dataPath = os.path.join(path_dir, '*.pkl')
    return glob.glob(dataPath)  # care about the serialization


files = get_files(path_dir = '../Data/Pickle_files')

In [ ]:
#%% Only when required to change the data-source
files[0:8], files[8:16] = files[8:16], files[0:8]
files

In [5]:
#%% Dataset Loader (final format)
import pickle

def data_set_loader(files):
    temp, dataSet, concat_dataset, data_loader = [], [], [], []

    import pdb

    for i in range(8):

        with open(files[i], 'rb') as f:
            temp.append(pickle.load(f))

        dataSet.append(data_set_Prep(temp[i], mode_no = 2, samp_siz= 16))
        
    concat_dataset = torch.utils.data.ConcatDataset(dataSet)
    data_loader = DataLoader(concat_dataset, batch_size=1, shuffle=True, num_workers=0, pin_memory=True)
    
    return data_loader

data_loader = []
del(data_loader)
data_loader = data_set_loader(files[0:8])

In [6]:
# temp[1][2].dtype
# resizing videos 
from torchvision import transforms
from torchvision.transforms import (
    RandomCrop,
    RandomResizedCrop,
)


class ReSizeVid():
    def __init__(self, size = (256, 256)):
        self.size = size
    
    def __call__(self, t_v):
        ll =  t_v.shape
        kk = torch.empty((ll[0], ll[1], ll[2], ll[3], self.size[0], self.size[1]))
        for i in range(ll[1]):
            kk[:, i,:,:,:,:] =  self.resize(t_v[0,i], self.size)
        return kk
    
    def resize(self, clip, target_size, interpolation_mode =  "bilinear"):
        assert len(target_size) == 2, "target size should be tuple (height, width)"
        return torch.nn.functional.interpolate(
            clip, size=target_size, mode=interpolation_mode
        )

trx = transforms.Compose([ReSizeVid((160, 160))])

In [9]:
#%% loading pretrain weight for the P3D199

def load_wt(net, pretrained_file='p3d_rgb_199.checkpoint.pth.tar'):
    weights=torch.load(pretrained_file)
    net.load_state_dict(weights)

In [8]:
from p3D_net import P3D131, P3D199, P3D63
from torchsummary import summary
import torch.nn as nn
#%% Input Model name
def get_model():
    ModName = input("Input your model name C3D, P3D199, P3D63, P3D131 \n")

    def str_to_class(ModelName):
        return getattr(sys.modules[__name__], ModelName)

    ModelName = str_to_class(ModName)

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(device)
    if ModName == 'C3D':
        net = ModelName()
        net.load_state_dict(torch.load('../../Saved_models/c3d.pickle'))
        net = net.cuda()
        input_shape = (3,16,112,112)
        print(summary(net, input_shape))

    elif ModName == 'P3D199':
        net = ModelName(False, 'RGB',num_classes=400)
        net = net.cuda()
        input_shape = (3,16,160,160)
        print(summary(net, input_shape))

    elif ModName == 'P3D63' or ModName == 'P3D131':
        net = ModelName(num_classes=400)
#         net.to(device)
#         input_shape = (3,16,160,160)
#         print(summary(net, input_shape))
        
    return net 

    #more network here!!
    #SLOWFAST
class L2Norm(nn.Module):
    def forward(self, x):
        return x / x.norm(p=2, dim=1, keepdim=True)



In [ ]:
import torch.nn as nn
def get_R_model(num_classes = 100, mlp = 0): # single layer only can add many layers if wanted

    net = get_model()
    # forced layer removal
    dim_layer = 2048
    net.fc =torch.nn.Linear(in_features=dim_layer, out_features= dim_layer)
    
    if mlp: 
        my_net = torch.nn.Sequential(net, nn.ReLU(), nn.Linear(dim_layer, num_classes), L2Norm() )
        
    else:
        net.fc =torch.nn.Linear(in_features=2048, out_features= num_classes)
        my_net = torch.nn.Sequential(net, nn.ReLU())
    return my_net
    
    
my_net = get_R_model(10, mlp = 0)
my_net.cuda()

In [22]:
def del_net(): ## Delete network 
    del(my_net)
    torch.cuda.empty_cache()
    return

In [ ]:
    
#%% SimCLR Alternate training loop from dataloader
#%% https://kozodoi.me/python/deep%20learning/pytorch/tutorial/2021/02/19/gradient-accumulation.html


L= NT_Xent(batch_size=5, temperature=0.05, world_size=1)

def simCLR_training(criterion, my_net, data_loader, lr= 0.5e-3):
    optimizer = torch.optim.SGD(my_net.parameters(), lr= lr) # very relevant params
    L= criterion.cuda()
    # tdc =  test_data(temp)
    j = 0
    my_net.train()
    for s1,s2 in data_loader: 
        s1, s2 =  trx(s1), trx(s2)
        s1, s2 =  s1[0].cuda(), s2[0].cuda()
        o1, o2 = my_net(s1),  my_net(s2)
        loss1 = L(o1, o2)
        loss1.backward()
        optimizer.step() 
   
        if j%500==0:
            print(loss1.cpu())
            # pdb.set_trace()
        j= j+1
        optimizer.zero_grad()   # zero the gradient buffers
    

simCLR_training(L, my_net, data_loader, lr = 1e-3)

In [ ]:
#%% Uniform - Alignment loss

import pdb
def al_un_training(criterion, my_net, data_loader, lr= 0.5e-3, lw =[0.9, 0.1]):
    optimizer = torch.optim.SGD(my_net.parameters(), lr= lr) # very relevant params
    L= criterion.cuda()
    # tdc =  test_data(temp)
    j = 0
    my_net.train()
    for s1,s2 in data_loader: 
        s1, s2 =  trx(s1), trx(s2)
        s1, s2 =  s1[0].cuda(), s2[0].cuda()
        o1, o2 = my_net(s1),  my_net(s2)
        align_loss_val = align_uniform.align_loss(o1,o2)
        uniform_loss_val1 = align_uniform.uniform_loss(o1, t = 2)
        uniform_loss_val2 = align_uniform.uniform_loss(o2, t = 2)

        loss1 = lw[0]*align_loss_val + lw[1]*uniform_loss_val1 + lw[1]*uniform_loss_val2
        
        loss1.backward()
        optimizer.step() 
        optimizer.zero_grad()   # zero the gradient buffers
#         pdb.set_trace()
        if j%500==0:
            print(loss1.cpu())
            # pdb.set_trace()
        j= j+1
        
    

al_un_training(L, my_net, data_loader, lr = 2e-3, lw =[10, 100])

In [ ]:
#%% InfoNCE Alternate training loop from dataloader
L= InfoNCE(temperature=0.05)
# tdc =  test_data(temp)


def infoNCE_training(criterion, my_net, data_loader, lr= 0.5e-3, bs  =1):
    j = 0
    optimizer = torch.optim.SGD(my_net.parameters(), lr= lr) # very relevant params
    L = criterion.cuda()
    my_net.train()
    for s1 in data_loader: 
        s1 =  trx(s1)
        s1 =  s1[0].cuda()
        output = my_net(s1)
        loss1 = L(output[0:1], output[1:2], output[2:])
        
        loss1 =  loss1/bs
        loss1.backward()
        
        if ((j+1)%bs == 0) or (j+1 ==len(data_loader)):
            optimizer.step() 
            optimizer.zero_grad()   # zero the gradient buffers
            
            
        if j%500==0:
            print(loss1.cpu()*bs)
            # pdb.set_trace()
        j= j+1
        
        
for _ in range(1):
    infoNCE_training(L, my_net, data_loader, lr = 1e-3, bs =1)

In [ ]:
torch.nn
with torch.no_grad():
    ot = my_net(s1)

In [39]:
files[9]

'../Data/Pickle_files/nirandi_8_26_21_300_300.pkl'

In [18]:
temp = []
with open(files[7], 'rb') as f:
            temp.append(pickle.load(f))

In [ ]:


#%% Preparing test dataset (change for every new dataset)
# test result
# a = np.int16([0, 1650 , 3240, 4530, 6240,7980,9180,10980, 11580,12030])

def embed_test(td, a, my_net):
    data_t =[]
    data_lab = []

    my_net.eval()
    for i in range(a.shape[0]-2):
        for _ in range(100):
            sample = torch.from_numpy(td.get_data_test(randint(a[i], a[i+1])))
            sample = trx(sample[None, :])[0]
            sample = sample.cuda()
            with torch.no_grad():
                output = my_net(sample)
            data_t.append(output.cpu().numpy())
            data_lab.append([i,i,i])


    data_t = np.array(data_t)
    data_lab = np.int16(data_lab)
    h,w,l = data_t.shape
    data_t = data_t.reshape((h*w,l))
    data_lab = data_lab.reshape((h*w,1))
    
    return data_t, data_lab, h, w

td = test_data(temp[0], frm_no=16)

a = np.int16([0, 27*30 , 58*30, 80*30, 110*30, 150*30, 175*30, 227*30, 257*30,325*30, 355*30])
named_labe =  ['sitting', 'standing', 'walking', 'Lying up', 'lying down', 'Obj inter', 'push up', 'leg move', 'hand wave, stand', 'what']


data_t, data_lab, h, w = embed_test(td, a, my_net)

# rmsev=  data_t/np.sqrt(np.sum(data_t**2, axis = 1))[:, np.newaxis] # 

In [ ]:
from sklearn.manifold import TSNE

def tsne_plot(data = data_t, n_comp = 2, label1 = label3, Lol = None, LoL = 1):
    if Lol== None:
        X_embedded = TSNE(n_components=n_comp, verbose=1).fit_transform(data)
    else:
        X_embedded = LoL
            
    fig = plt.figure()
    ax = fig.add_subplot()
    ax.set_aspect('equal', adjustable='box')
    if n_comp == 3:ax = fig.add_subplot(projection ='3d')
    
    # cdict = {0: 'red', 1: 'blue', 2: 'green'}
    
    markers = ['v', 'x', 'o', '.', '>', '<', '1', '2', '3']
    
    for i, g in enumerate(np.unique(label1)):
        ix = np.where(label1 == g)
        if n_comp==3:
            ax.scatter(X_embedded[ix,0], X_embedded[ix,1], X_embedded[ix,2], marker = markers[i], label = g, alpha = 0.8)
        else:
            ax.scatter(X_embedded[ix,0], X_embedded[ix,1], marker = markers[i], label = g, alpha = 0.8)
    
    ax.set_xlabel('X Label')
    ax.set_ylabel('Y Label')
    if n_comp==3:ax.set_zlabel('Z Label')
    if n_comp==3:ax.set_zlabel('Z Label')

    ax.legend(fontsize='large', markerscale=2); plt.show()
    ax.legend(fontsize='large', markerscale=2); plt.show()
    #plt 2
    fig = plt.figure()
    ax = fig.add_subplot()
    if n_comp == 3:ax = fig.add_subplot(projection ='3d')
    
    # cdict = {0: 'red', 1: 'blue', 2: 'green'}
    markers = ['v', 'x', 'x', '.', '>', '<', '1', '2', '3', '4']
    for i, g in enumerate([3,2]):# enumerate(np.unique(data_lab)):
        ix = np.where(data_lab == g)
        if n_comp==3:
            ax.scatter(X_embedded[ix,0], X_embedded[ix,1], X_embedded[ix,2], marker = markers[i], label = g, alpha = 0.8)
        else:
            ax.scatter(X_embedded[ix,0], X_embedded[ix,1], marker = markers[i], label = named_labe[g], alpha = 0.8)
    ax.set_xlabel('X Label')
    ax.set_ylabel('Y Label')
    ax.legend(fontsize='large', markerscale=2, bbox_to_anchor=(0.5, .5, 1.2, 0.5))
    ax.set_aspect('equal', adjustable='box')
    plt.show()
    

label3 = np.reshape(np.int16([ i for i in range(h*w)])%3, (h*w,1))
tsne_plot(data_t, 2, label3, Lol =None,  LoL = 1)

# shillohette coefficient! 

In [22]:
#%% Model weight save and load

torch.save(my_net.state_dict(), '../My_net_p_131_save.pth')

load_wt(my_net, pretrained_file='../My_net_p_131_save.pth')